In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    confusion_matrix,
    cohen_kappa_score,
    classification_report,
    accuracy_score,
    roc_auc_score,
)
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
class CharsCountTransformer(BaseEstimator, TransformerMixin):
    def get_relevant_chars(self, tweet):
        num_hashtags = tweet.count("#")
        num_exclamations = tweet.count("!")
        num_interrogations = tweet.count("?")
        num_at = tweet.count("@")
        return [num_hashtags, num_exclamations, num_interrogations, num_at]

    def transform(self, X, y=None):
        chars = []
        for tweet in X:
            chars.append(self.get_relevant_chars(tweet))

        return np.array(chars)

    def fit(self, X, y=None):
        return self

In [30]:
def auc_score(test_set, predicted_set):
    high_predicted = np.array([prediction[2] for prediction in predicted_set])
    medium_predicted = np.array(
        [prediction[1] for prediction in predicted_set]
    )
    low_predicted = np.array([prediction[0] for prediction in predicted_set])
    inc_test = np.where(test_set == "incivilidad", 1.0, 0.0)
    odio_test = np.where(test_set == "odio", 1.0, 0.0)
    normal_test = np.where(test_set == "normal", 1.0, 0.0)
    auc_high = roc_auc_score(inc_test, high_predicted)
    auc_med = roc_auc_score(odio_test, medium_predicted)
    auc_low = roc_auc_score(normal_test, low_predicted)
    auc_w = (
        normal_test.sum() * auc_low
        + odio_test.sum() * auc_med
        + inc_test.sum() * auc_high
    ) / (normal_test.sum() + odio_test.sum() + inc_test.sum())
    return auc_w


def evaluate(predicted_probabilities, y_test, labels):
    # Importante: al transformar los arreglos de probabilidad a clases,
    # entregar el arreglo de clases aprendido por el clasificador.
    # (que comunmente, es distinto a ['normal', 'odio', 'incivilidad'])
    predicted_labels = [
        labels[np.argmax(item)] for item in predicted_probabilities
    ]

    print("Matriz de confusión")
    print(
        confusion_matrix(
            y_test, predicted_labels, labels=["normal", "odio", "incivilidad"]
        )
    )

    print("\nReporte de clasificación:\n")
    print(
        classification_report(
            y_test, predicted_labels, labels=["normal", "odio", "incivilidad"]
        )
    )
    # Reorder predicted probabilities array.
    labels = labels.tolist()

    predicted_probabilities = predicted_probabilities[
        :,
        [
            labels.index("normal"),
            labels.index("odio"),
            labels.index("incivilidad"),
        ],
    ]

    auc = round(auc_score(y_test, predicted_probabilities), 3)
    print("Métricas:\n\nAUC: ", auc, end="\t")
    kappa = round(cohen_kappa_score(y_test, predicted_labels), 3)
    print("Kappa:", kappa, end="\t")
    accuracy = round(accuracy_score(y_test, predicted_labels), 3)
    print("Accuracy:", accuracy)
    print("------------------------------------------------------\n")
    return np.array([auc, kappa, accuracy])


In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/new/assignment_1/train/train.tsv", sep="\t")

In [11]:
stopwords = nltk.corpus.stopwords.words('spanish')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    data.texto,
    data.clase,
    shuffle=True,
    test_size=0.33,
    random_state=42,
)


In [22]:
vectorizer = TfidfVectorizer(
    stop_words=stopwords,
)
ml_pipeline = Pipeline(
    [
        (
            "features",
            FeatureUnion(
                [
                    ("vectorizer", vectorizer),
                    ("chars_count", CharsCountTransformer()),
                ]
            ),
        ),
        ("clf", RandomForestClassifier(n_estimators=800, n_jobs=-1)),
    ]
)


In [38]:
param_grid = {
    "clf__n_estimators": [100,300,800,1000],
    "clf__criterion":['gini', 'entropy', 'log_loss']
}
search = GridSearchCV(
    ml_pipeline,
    param_grid,
    n_jobs=-1
)
search.fit(X_train, y_train)

: 

: 

In [ ]:
search.best_params_

In [23]:
ml_pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('vectorizer',
                                                 TfidfVectorizer(stop_words=['de',
                                                                             'la',
                                                                             'que',
                                                                             'el',
                                                                             'en',
                                                                             'y',
                                                                             'a',
                                                                             'los',
                                                                             'del',
                                                                             'se',
                                                                             

In [32]:
predicted_probabilities = ml_pipeline.predict_proba(X_test)
learned_labels = ml_pipeline.classes_
scores = evaluate(predicted_probabilities, y_test, learned_labels)

Matriz de confusión
[[1106   63  288]
 [ 216  424  197]
 [ 228   26 1483]]

Reporte de clasificación:

              precision    recall  f1-score   support

      normal       0.71      0.76      0.74      1457
        odio       0.83      0.51      0.63       837
 incivilidad       0.75      0.85      0.80      1737

    accuracy                           0.75      4031
   macro avg       0.76      0.71      0.72      4031
weighted avg       0.75      0.75      0.74      4031

Métricas:

AUC:  0.897	Kappa: 0.595	Accuracy: 0.747
------------------------------------------------------



In [33]:
scores

array([0.897, 0.595, 0.747])